In [1]:

import torch.nn as nn    
    
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from torchvision import transforms
import os
from torch.optim import Adam
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('Agg')
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from torch.nn.functional import cosine_similarity

In [2]:

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Define the CNN backbone (shared for both inputs)
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 16, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2)
        )

        # Fully connected layer for similarity
        self.fc = nn.Sequential(
            nn.Linear(144, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 64)
           # Binary classification: Similar (1) or Not Similar (0)
        )

    def forward_one(self, x):
        # Forward pass for one input
        x = self.backbone(x)
        
        x = x.view(x.size()[0], -1)
        #print('x',x.size())
        x=self.fc(x) # Flatten
        return x

    def forward(self, input1, input2):
        # Forward pass for both inputs
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        
        #print('out1',output1.size())
        
        # Compute the cosine similarity between the two embeddings.
        cos_sim = cosine_similarity(output1, output2, dim=1, eps=1e-6)
        cos_sim = cos_sim.unsqueeze(1)  # Add an extra dimension to match the shape required for the fc layer.
        #print('cos_sim',cos_sim,cos_sim.size())
        # Similarity prediction
        #similarity = self.fc(cos_sim)
        prob_sim = torch.sigmoid(cos_sim)
        return prob_sim

    def get_embedding(self, x):
        # Method to get the embedding of a single image
        return self.forward_one(x)
    
class CellTrackingDataset(Dataset):
    def __init__(self, root_dir, window,transform=None):
        # Load all TXT files in root_dir into a list of dataframes
        all_dfs = []
        for folder in os.listdir(root_dir):
            print('folder',folder)
            # Check if the folder's name ends with '_GT'
            if folder.endswith("_GT"):#folder=='1_GT': #!!!!!!!!
                file_path = os.path.join(root_dir, folder, 'TRA','pos_GT.txt')
                print('file_path',file_path)
                # Check if 'pos_GT.txt' actually exists in the folder
                if os.path.exists(file_path):
                    print('file', file_path)
                    run = int(folder.split('_')[0])  # extract run number from folder name
                    print('run', run)
                    df = pd.read_csv(file_path, delimiter='\t', skiprows=1, 
                                     names=['x', 'y', 'r', 'id', 'split_id', 's_prob', 't'])
                    df['run'] = run  # Add a run column
                    all_dfs.append(df)

        self.data = pd.concat(all_dfs, ignore_index=True)
        self.root_dir = root_dir
        self.transform = transform
        self.window=window

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # TODO: Implement logic to decide positive or negative pair
        # For example, 50% of the time it returns positive pairs, and 50% negative pairs
        positive_sample = np.random.choice([True, False])

        if positive_sample:
            # Load two consecutive frames of the same cell
            #print("IDX:", idx, "DATA LENGTH:", len(self.data), self.data.iloc[idx])
            frame_t, frame_t_plus_1 = self._load_consecutive_frames(self.data.iloc[idx])
            label = torch.tensor([1.0])
        else:
            # Load frames of two different cells (from consecutive frames)
            #print("IDX:", idx, "DATA LENGTH:", len(self.data), self.data.iloc[idx])
            frame_t, frame_t_plus_1 = self._load_different_cells_frames(self.data.iloc[idx])
            label = torch.tensor([0.0])

        return (frame_t, frame_t_plus_1), label

    def _load_consecutive_frames(self, row):
        # Find the next frame's cell that has either the same 'id' 
        # or has 'split_id' matching the current 'id'
        if row['id'] ==-1 or row['id']==-2:
            same_cell_or_daughter = self.data[(self.data['run'] == row['run']) & 
                                          ((self.data['id'] == row['id']) | 
                                           (self.data['split_id'] == row['id'])|
                                           (self.data['id'] == row['split_id']))]
        else:
            same_cell_or_daughter = self.data[(self.data['t'] == row['t'] + 1) & 
                                              (self.data['run'] == row['run']) & 
                                              ((self.data['id'] == row['id']) | 
                                               (self.data['split_id'] == row['id']))]

        if not same_cell_or_daughter.empty:
            next_frame_same_cell = same_cell_or_daughter.sample(n=1).iloc[0]
            frame_t_plus_1 = self._load_frame(next_frame_same_cell)
        else:
            # Handle the scenario when no such cell exists in the next frame
            frame_t_plus_1 = torch.zeros_like(self._load_frame(row))  # Placeholder, modify as needed

        frame_t = self._load_frame(row)

        return frame_t, frame_t_plus_1

    def _load_different_cells_frames(self, row):
        # Find another cell from the next frame but within the same run 
        # and is not a daughter cell (i.e., its 'split_id' doesn't match the current cell's 'id')
        
        if row['id'] ==-1:
            next_frame_data = self.data[(self.data['run'] == row['run']) & 
                                        (self.data['split_id'] != row['id'])]
        else:
             next_frame_data = self.data[(self.data['t'] == row['t'] + 1) & 
                                        (self.data['run'] == row['run']) & 
                                        (self.data['split_id'] != row['id'])]
            

        # Ensure that you're getting a different cell, but from the same run
        different_cell = next_frame_data[next_frame_data['id'] != row['id']]

        if not different_cell.empty:
            different_cell_row = different_cell.sample(n=1).iloc[0]
            frame_t_plus_1 = self._load_frame(different_cell_row)
        else:
            # Handle the scenario when no different cell exists in t+1 frame
            frame_t_plus_1 = torch.zeros_like(self._load_frame(row))  # Placeholder, modify as needed

        frame_t = self._load_frame(row)

        return frame_t, frame_t_plus_1
    def _load_frame(self, row):
        # Extract position and run from the row
        pos_x, pos_y, run, t = row['x'], row['y'], row['run'], row['t']
        # Using the fingerprint_cnn function to get the tensor
        im_tens = fingerprint_cnn(pos_x, pos_y, self.window, t, run,path0=self.root_dir)
        return im_tens
    
def fingerprint_cnn(pos_x,pos_y,window,t,run,void_node=False,a_x=113.89266922,b_x=19.51478307,a_y=-114.14194117,b_y=131.3404747,path0='.'):
    v_flag=False
    if pos_x==2:
        #print('void')
        #pos_x = 0.5
        #pos_y = 0.5
        
        


        pathm1=path0+'void.png'
        path=path0+'void.png'
        pathp1=path0+'void.png'
        v_flag=True
        
    else:
        pathm1=path0+str(int(run))+'/'+str(int(t-1))+'.png'
        path=path0+str(int(run))+'/'+str(int(t))+'.png'
        pathp1=path0+str(int(run))+'/'+str(int(t+1))+'.png'


    x_l,x_r,y_l,y_u=calibrate_pix_pos(pos_x,pos_y,a_x,b_x,a_y,b_y,window)


       


    im = Image.open(path).convert('L')
    shape = np.asarray(im).shape
    try:
        im_m = Image.open(pathm1).convert('L')
    except:
        pix=255*np.ones(shape)
        im_m = Image.fromarray(pix).convert('L')
    try:
        im_p = Image.open(pathp1).convert('L')
    except:
        pix=255*np.ones(shape)
        im_p = Image.fromarray(pix).convert('L')




    if v_flag:
        im_crop = im
        im_m_crop = im_m
        im_p_crop = im_p
    else:
        im_crop = im.crop((x_l, y_l, x_r, y_u))
        im_m_crop = im_m.crop((x_l, y_l, x_r, y_u))
        im_p_crop = im_p.crop((x_l, y_l, x_r, y_u))






    #if t==9 and run==95:
    #    im_crop.show()


    im_crop = im_crop.resize((20,20),Image.ANTIALIAS)
    im_m_crop = im_m_crop.resize((20,20),Image.ANTIALIAS)
    im_p_crop = im_p_crop.resize((20,20),Image.ANTIALIAS)

    imarray = np.asarray(im_crop)
    #print('shape',imarray.shape)
    im_m_array = np.asarray(im_m_crop)
    im_p_array = np.asarray(im_p_crop)


    im_tens=torch.zeros(3, 20,20)


    convert_tensor = transforms.ToTensor()
    im_tens[1]=convert_tensor(imarray)
    im_tens[0]=convert_tensor(im_m_array)
    im_tens[2]=convert_tensor(im_p_array)
    
    #if v_flag:
    #    plt.imshow(im_tens.numpy().transpose(1, 2, 0))

    return im_tens

def calibrate_pix_pos(pos_x,pos_y,a_x,b_x,a_y,b_y,window):
    #pix_x0=int(fit_func(pos_x,a_x,b_x))
    #pix_y0 = pos_x* 512
    #pix_x0 = (1-pos_y) * 512
    pix_x0 = pos_x* 512
    pix_y0 = (1-pos_y) * 512
    #pix_y0=int(fit_func(pos_y,a_y,b_y))
    #print(pix_x0,pix_y0,int(pix_x0-window))
    return int(pix_x0-window),int(pix_x0+window),int(pix_y0-window),int(pix_y0+window)


def imshow(img1, img2, label):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    # Assuming images are normalized pytorch tensors, convert them to numpy and de-normalize if necessary
    ax1.imshow(img1[0].numpy().transpose(1, 2, 0))
    ax2.imshow(img2[0].numpy().transpose(1, 2, 0))
    
    # Display the label
    fig.suptitle(f"Label: {label.item()}")
    plt.show()
    

def fit_func(x,a,b):
    return a*x+b


def extract_embeddings_subset(dataloader, model):
    model.eval()
    
    embeddings = np.zeros((20, 64))
    labels = np.zeros(20)
    k = 0
    positive_count, negative_count = 0, 0

    with torch.no_grad():
        for (input1, input2), batch_targets in dataloader:
            for idx, label in enumerate(batch_targets):  # Iterate through each label in the batch
                if label.item() == 1.0 and positive_count < 5:
                    embeddings[k] = model.get_embedding(input1[idx].unsqueeze(0)).data.cpu().numpy()
                    embeddings[k+1] = model.get_embedding(input2[idx].unsqueeze(0)).data.cpu().numpy()
                    labels[k] = 1.0
                    labels[k+1] = 1.0
                    k += 2
                    positive_count += 1

                elif label.item() == 0.0 and negative_count < 5:
                    embeddings[k] = model.get_embedding(input1[idx].unsqueeze(0)).data.cpu().numpy()
                    embeddings[k+1] = model.get_embedding(input2[idx].unsqueeze(0)).data.cpu().numpy()
                    labels[k] = 0.0
                    labels[k+1] = 0.0
                    k += 2
                    negative_count += 1

                if positive_count >= 5 and negative_count >= 5:
                    break

    return embeddings, labels

def plot_embeddings(embeddings, labels, title=''):
    plt.figure(figsize=(10, 10))
    colors = ['blue' if label == 1.0 else 'red' for label in labels]
    
    for i in range(0, len(labels), 2):  # We are skipping every second one because we know they come in pairs
        plt.plot(embeddings[i:i+2, 0], embeddings[i:i+2, 1], '-o', c=colors[i], mfc='white') # Line between pair
        
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=colors, s=60, edgecolors='w')
    plt.title(title)
    plt.show()

def visualize_tsne_pca(dataloader, model):
    embeddings, labels = extract_embeddings_subset(dataloader, model)
    embeddings_tsne = TSNE(n_components=2, perplexity=5).fit_transform(embeddings)  # Adjusted perplexity here
    embeddings_pca = PCA(n_components=2).fit_transform(embeddings)
    plot_embeddings(embeddings_tsne, labels, title="t-SNE visualization of embeddings")
    plot_embeddings(embeddings_pca, labels, title="PCA visualization of embeddings")

def train_one_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for (input1, input2), labels in dataloader:
        #imshow(input1, input2, labels[0])
        optimizer.zero_grad()

        outputs = model(input1, input2).squeeze()
        #print('out',outputs)

        predicted_labels = (outputs > 0.5).float()
        correct_predictions += (predicted_labels == labels.squeeze()).float().sum().item()
        total_samples += labels.size(0)
        outputs = torch.clamp(outputs, min=1e-7, max=1-1e-7)
        #print('out',outputs)
        
        loss = F.binary_cross_entropy(outputs, labels.squeeze())

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    epoch_accuracy = 100 * correct_predictions / total_samples
    avg_loss = total_loss / len(dataloader)
    return avg_loss, epoch_accuracy

def validate(model, dataloader):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for (input1, input2), labels in dataloader:
            outputs = model(input1, input2).squeeze()
            predicted_labels = (outputs > 0.8).float()
            correct_predictions += (predicted_labels == labels.squeeze()).float().sum().item()
            total_samples += labels.size(0)
            outputs = torch.clamp(outputs, min=1e-7, max=1-1e-7)
            loss = F.binary_cross_entropy(outputs, labels.squeeze())
            total_loss += loss.item()
            
    epoch_accuracy = 100 * correct_predictions / total_samples
    avg_loss = total_loss / len(dataloader)
    return avg_loss, epoch_accuracy

def main():
    win = 115
    root_dir_train = '/media/mo/Label/HeLa_2_track_train/'
    train_dataset = CellTrackingDataset(root_dir_train,window=win)
    root_dir_test = '/media/mo/Label/HeLa_2_track_test/'
    test_dataset = CellTrackingDataset(root_dir_test,window=win)
    
    # Split the dataset into train and test

    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = SiameseNetwork()
    model.load_state_dict(torch.load('Siam_cell_new.pt'))
    optimizer = Adam(model.parameters(), lr=0.001)

    num_epochs = 10
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train_one_epoch(model, train_dataloader, optimizer)
        val_loss,test_accuracy = validate(model, test_dataloader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Test Accuracy:{test_accuracy:.2f}%")

        # Save checkpoint with minimal validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'Siam_cell_new.pt')
    
# Usage:
#data = ... # Load your ground truth data here, for instance as a Pandas DataFrame
# '/home/mo/Desktop/IWR/TimeGraph2.0/dot_folder/'
#import os

#for folder in os.listdir(root_dir):
#    print(folder, os.listdir(os.path.join(root_dir, folder)))


print(stop)

NameError: name 'stop' is not defined


root_dir = '/media/mo/Label/HeLa_track/'

dataset = CellTrackingDataset(root_dir)
model = SiameseNetwork()
optimizer = Adam(model.parameters(), lr=0.001)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for (input1, input2), labels in dataloader:
        #print(input1.shape)
        imshow(input1, input2, labels[0])
        optimizer.zero_grad()
        
        # Forward pass
        
        outputs = model(input1, input2).squeeze()
        
        # Convert the continuous valued outputs to binary predictions
        predicted_labels = (outputs > 0.5).float()
        correct_predictions += (predicted_labels == labels.squeeze()).float().sum().item()
        total_samples += labels.size(0)
        
        # Compute loss
        loss = F.binary_cross_entropy(outputs, labels.squeeze())
        
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    epoch_accuracy = 100 * correct_predictions / total_samples
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}, Accuracy: {epoch_accuracy:.2f}%")
torch.save(model.state_dict(), 'Siam_cell.pt')
visualize_tsne_pca(dataloader, model)

In [3]:
main()

folder 10
folder 19
folder 29
folder 38
folder 61
folder 1
folder 10_GT
file_path /media/mo/Label/HeLa_2_track_train/10_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/10_GT/TRA/pos_GT.txt
run 10
folder 11
folder 11_GT
file_path /media/mo/Label/HeLa_2_track_train/11_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/11_GT/TRA/pos_GT.txt
run 11
folder 12
folder 12_GT
file_path /media/mo/Label/HeLa_2_track_train/12_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/12_GT/TRA/pos_GT.txt
run 12
folder 13
folder 13_GT
file_path /media/mo/Label/HeLa_2_track_train/13_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/13_GT/TRA/pos_GT.txt
run 13
folder 14
folder 14_GT
file_path /media/mo/Label/HeLa_2_track_train/14_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/14_GT/TRA/pos_GT.txt
run 14
folder 15
folder 15_GT
file_path /media/mo/Label/HeLa_2_track_train/15_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/15_GT/TRA/pos_GT.txt
run 15
folder vo

file /media/mo/Label/HeLa_2_track_train/74_GT/TRA/pos_GT.txt
run 74
folder 75
folder 75_GT
file_path /media/mo/Label/HeLa_2_track_train/75_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/75_GT/TRA/pos_GT.txt
run 75
folder 77
folder 77_GT
file_path /media/mo/Label/HeLa_2_track_train/77_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/77_GT/TRA/pos_GT.txt
run 77
folder 78
folder 78_GT
file_path /media/mo/Label/HeLa_2_track_train/78_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/78_GT/TRA/pos_GT.txt
run 78
folder 79
folder 79_GT
file_path /media/mo/Label/HeLa_2_track_train/79_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/79_GT/TRA/pos_GT.txt
run 79
folder 7_GT
file_path /media/mo/Label/HeLa_2_track_train/7_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/7_GT/TRA/pos_GT.txt
run 7
folder 8
folder 80
folder 80_GT
file_path /media/mo/Label/HeLa_2_track_train/80_GT/TRA/pos_GT.txt
file /media/mo/Label/HeLa_2_track_train/80_GT/TRA/pos_GT.txt
ru

/tmp/ipykernel_149974/947111407.py:219: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im_crop = im_crop.resize((20,20),Image.ANTIALIAS)
/tmp/ipykernel_149974/947111407.py:220: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im_m_crop = im_m_crop.resize((20,20),Image.ANTIALIAS)
/tmp/ipykernel_149974/947111407.py:221: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im_p_crop = im_p_crop.resize((20,20),Image.ANTIALIAS)
/home/mo/anaconda3/lib/python3.10/site-packages/torchvision/transforms/functional.py:152: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data o

Epoch 1/10, Train Loss: 0.5655, Train Accuracy: 73.87%, Val Loss: 0.5593, Test Accuracy:50.12%
Epoch 2/10, Train Loss: 0.5674, Train Accuracy: 73.49%, Val Loss: 0.5685, Test Accuracy:47.60%
Epoch 3/10, Train Loss: 0.5651, Train Accuracy: 73.91%, Val Loss: 0.5711, Test Accuracy:52.58%
Epoch 4/10, Train Loss: 0.5646, Train Accuracy: 74.09%, Val Loss: 0.5559, Test Accuracy:49.46%
Epoch 5/10, Train Loss: 0.5610, Train Accuracy: 74.32%, Val Loss: 0.5595, Test Accuracy:51.80%
Epoch 6/10, Train Loss: 0.5649, Train Accuracy: 73.93%, Val Loss: 0.5701, Test Accuracy:50.66%
Epoch 7/10, Train Loss: 0.5628, Train Accuracy: 73.69%, Val Loss: 0.5778, Test Accuracy:51.74%
Epoch 8/10, Train Loss: 0.5624, Train Accuracy: 74.63%, Val Loss: 0.5570, Test Accuracy:51.56%
Epoch 9/10, Train Loss: 0.5543, Train Accuracy: 75.79%, Val Loss: 0.5558, Test Accuracy:49.76%
Epoch 10/10, Train Loss: 0.5504, Train Accuracy: 76.05%, Val Loss: 0.5697, Test Accuracy:52.64%


In [ ]:
model = SiameseNetwork()
# 2. Load the state dictionary
model.load_state_dict(torch.load('Siam_cell_best.pt'))
# 3. If you're doing inference, set the model to evaluation mode
model.eval()


dataset = CellTrackingDataset('/media/mo/Label/HeLa_track_test/') 
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
visualize_tsne_pca(dataloader, model)